In [ ]:
from selenium import webdriver
import base64, pickle, time, os
from IPython.display import Image
from IPython.display import display
from PIL import Image as pil
from selenium.webdriver.chrome.options import Options

### 1. Get Youtube Images

##### (1) Get Youtube Driver & Save Element Screen

In [ ]:
def getDriver(youtube_url):
    chrome_options = Options()
    chrome_options.add_argument("--window-size=1920,1080")
    driver = webdriver.Chrome(chrome_options=chrome_options)
#    driver =  webdriver.Chrome()
    driver.get(youtube_url)
    return driver

def saveScreen(driver, filename):
    element = driver.find_element_by_css_selector('#movie_player')
    location = element.location
    size = element.size
    driver.save_screenshot('screenshot.png')
    im = pil.open('screenshot.png')
    
    left = location['x']
    top = location['y']
    right = left + size['width']
    bottom = top + size['height']

    # left, top, right, bottom
    im = im.crop((left, top, right, bottom)) 
    im.save(filename) 

##### (2) Check Element

In [ ]:
def checkElement(driver, selector):
    result = True
    try:
        driver.find_element_by_css_selector(selector)
    except:
        result = False
    return result

##### (3) Skip Ad

In [ ]:
skipAd_selector = ".videoAdUiSkipButton.videoAdUiAction.videoAdUiFixedPaddingSkipButton"
closeAd_selector = ".adDisplay.extra-padding .close-button"

def skipAd(driver):
    if checkElement(driver, skipAd_selector):
        driver.find_element_by_css_selector(skipAd_selector).click()
    
def closeAd(driver):
    if checkElement(driver, closeAd_selector):
        driver.find_element_by_css_selector(closeAd_selector).click()

##### (4) Make Directory

In [ ]:
def makeDir(youtube_url):
    dirList = os.listdir("data/images/")
    dirName = youtube_url.split("v=")[1]
    isDir = True
    if dirName not in dirList:
        os.makedirs("data/images/" + dirName)
        isDir = False
    return isDir, dirName

##### (5) Save Images

In [ ]:
def saveImages(youtube_url, start_term, image_count, save_term):
    
    # open driver
    driver = getDriver(youtube_url)

    # skip ad
    time.sleep(6)
    skipAd(driver)

    # save images
    time.sleep(start_term)
    for idx in range(1, image_count+1):
        saveScreen(driver, 'data/images/' + dirName + '/screenshot' + str(idx) + '.png')
        time.sleep(save_term)
        print('saved : screenshot' + str(idx) + '.png')
    # close driver
    driver.close()

##### (6) resize images

In [ ]:
def makeTunmnail(image_count, dirName):
# make image list
    filenames = []
    for idx in range(1, image_count + 1):
        filename = 'C:/Users/hjnam/study/crawling/data/images/' + dirName  + '/screenshot' + str(idx) + '.png'
        filenames.append(filename)

    # make thumnail
    size = (512, 512)
    thumnails = []
    for filename in filenames:
        im = pil.open(filename)
        im.thumbnail(size)
        savename = filename.split("/")
        savename[len(savename)-1] = savename[len(savename)-1].replace(".png","_thumbnail.png")
        savename = "/".join(savename)
        im.save(savename, "png")
        thumnails.append(savename)
        
    return thumnails

In [ ]:
youtube_url = "https://www.youtube.com/watch?v=xyq2T_K6DHg" # 이미지 생성 youtube URL 

# 도깨비 
# youtube_url = "https://www.youtube.com/watch?v=2kZg7byEYGA"

In [ ]:
%%time
isDir, dirName = makeDir(youtube_url)
start_term = 5 # 20초 후 부터 이미지 생성
image_count = 15 # 10개의 이미지 생성
save_term = 0.1 # 저장 텀
saveImages(youtube_url, start_term, image_count, save_term)

thumnails = makeTunmnail(image_count, dirName)

### 2. IBM Wasson API

In [ ]:
url = 'https://visual-recognition-demo.mybluemix.net/'

In [ ]:
def wasson(filenames):

    # open site
    driver =  webdriver.Chrome()
    driver.get(url)

    # input file
    driver.find_element_by_css_selector(".use--file-input-button-text")
      
    for filename in filenames:
    
        # file upload
        driver.find_element_by_id("use--file").send_keys(filename)

        resultData = {}

        # check done analytics
        for _ in range(30):
            if checkElement(driver, ".results-table--container"):
#                resultData = run()
                def makeData(element):
                    # make datas
                    results = element.find_elements_by_css_selector(".base--tr")        
                    data = {}
                    for result in results:
                        if checkElement(result, ".result--class"):
                            key = result.find_element_by_css_selector(".result--class").text
                            value = result.find_element_by_css_selector(".result--score .results-table--score-value").text
                            data[key] = value
                        else:
                            break
#                    print(data)
                    return data


                containers = driver.find_elements_by_css_selector(".results-table--container")

                for container in containers:

                    label = container.find_elements_by_css_selector(".base--thead .base--th.results-table--header")[0]    
                    elements = container.find_elements_by_css_selector("table tbody")

                    if label.text == "Faces":
                        data = []
                        for idx in range(len(elements) - 1):
                            data.append(makeData(elements[idx]))
                        resultData[label.text] = data
                    else: #label.text == "Classes" or label.text == "Food":
                        data = makeData(elements[0])
                        resultData[label.text] = data
                print(resultData)
                break
            else:
                time.sleep(1)

    driver.close()
    return resultData

In [ ]:
%%time
result = wasson(thumnails)